In [ ]:
pip install pygame

### Play around with this parameter $m$ or "nodes_repelling_now"

For context: Youtube video link

In [11]:
import pygame
import sys
import random
import math

nodes_repelling_now = 40  #parameter m (on my slides had 5 and 40)

# Set the number of nodes per group (white and grey)
number_per_group = [40, 40] 

# Set the dimensions of the simulation arena
WIDTH, HEIGHT = 400, 300

# Set the frames per second for the simulation
fps = 10

# Set the number of active and repelling nodes
nodes_active_now = 1

# First group has 20 members, second group has 20 members
number_per_group = [40, 40] 

# Set the duration for which each node remains active
duration = 3000  #Depending on your fps, this is how quickly the nodes will change.

# Set whether to add lines between nodes
add_line = True

# Set the maximum and minimum distances for normalization
max_ = 500
min_ = 20



# Initialize the time when the nodes were last updated
last_update_time = pygame.time.get_ticks()

# Define the Point class
class Point:
    def __init__(self, x, y, group, color):
        self.x = x
        self.y = y
        self.group = group
        self.color = color
        self.radius = 10
        self.dx = 0
        self.dy = 0

    # Method to draw the point on the screen
    def draw(self, is_group=False, is_active_or_target=False, is_repelling=False):
        # Set the default color for non-group points
        color = (128, 128, 128)
        # If the point is part of a group, set the color to white
        if is_group:
            color = (255, 255, 255)

        # Draw the point
        pygame.draw.circle(screen, color, (int(self.x), int(self.y)), self.radius)

        # If the point is active or a target, draw a green circle around it
        if is_active_or_target:
            pygame.draw.circle(screen, (0, 255, 0), (int(self.x), int(self.y)), self.radius, 2)

        # If the point is repelling, draw a red circle around it
        if is_repelling:
            pygame.draw.circle(screen, (255, 0, 0), (int(self.x), int(self.y)), self.radius, 2)

    # Method to calculate the attraction force to another point
    def attract(self, other):
        force, angle = attraction_force(self, other)
        fx = force * math.cos(angle)
        fy = force * math.sin(angle)
        return fx, fy

    # Method to calculate the repulsion force from another point
    def repel(self, other):
        force, angle = repulsion_force(self, other)
        fx = force * math.cos(angle)
        fy = force * math.sin(angle)
        return fx, fy

    # Method to calculate the distance to another point
    def distance_to(self, other):
        dx = self.x - other.x
        dy = self.y - other.y
        return math.sqrt(dx*dx + dy*dy)

    # Method to bounce off the walls of the arena
    def bounce(self, WIDTH, HEIGHT):
        if self.x < 0 or self.x > WIDTH:
            self.dx *= -1
        if self.y < 0 or self.y > HEIGHT:
            self.dy *= -1

# Function to calculate the attraction force between two points (used for calculation & plotting line widths)
def attraction_force(self, other):
    dx = other.x - self.x
    dy = other.y - self.y
    distance = math.sqrt(dx*dx + dy*dy)
    # Normalize the distance
    normalized_distance = (distance - min_)/(max_ - min_) 
    force =  math.exp(0.5 ** normalized_distance**2) -1 #in theory, closer points should have a higher attraction force,
    angle = math.atan2(dy, dx)                          #but since the points overshoot, I reduce force when they are closer
    return force, angle                             

# Function to calculate the repulsion force between two points (used for calculation & plotting line widths)
def repulsion_force(self, other):
    dx = other.x - self.x
    dy = other.y - self.y
    distance = math.sqrt(dx*dx + dy*dy)
    # Normalize the distance
    normalized_distance = (distance - min_)/(max_ - min_)
    
    force = (0.2 * math.exp(- normalized_distance**2))/nodes_repelling_now #control for the number of repelling nodes
    angle = math.atan2(dy, dx)
    return force, angle


# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Point Attraction and Repulsion")

# Define group colors
colors = [(255, 255, 255), (120, 120, 120)]  # White and grey

# Create random points
points = []

# For each group, create points and add them to the list
for i in range(2):    
    for _ in range(number_per_group[i]):
        points.append(Point(random.randint(0, WIDTH), random.randint(0, HEIGHT), i, colors[i]))

# Separate points into group and non-group points
group_points = [p for p in points if p.group == 0]  # White points
non_group_points = [p for p in points if p.group == 1]  # Grey points

# Initialize active and target points
active_point = random.choice(group_points)
target_point = random.choice(group_points)
repelling_points = random.sample(non_group_points, nodes_repelling_now)

# Main loop 
while True:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # Clear the screen
    screen.fill((0, 0, 0))

    # Initialize position changes
    position_active_point = [0, 0] # [x, y]

    # Calculate changes
    for point in group_points:
        # Update active and target points every 10 seconds
        current_time = pygame.time.get_ticks()
        if current_time - last_update_time > duration:
            active_point = random.choice(group_points)
            target_point = random.choice(group_points)
            repelling_points = random.sample(non_group_points, nodes_repelling_now)
            last_update_time = pygame.time.get_ticks()
            
        # Attraction to target point
        if point == target_point:
            fx, fy = point.attract(point)
            position_active_point[0] += fx
            position_active_point[1] += fy 

        # Repulsion from all points
        for other_point in points:
            if other_point in repelling_points:
                fx, fy = point.repel(other_point)
                position_active_point[0] -= fx
                position_active_point[1] -= fy

    # Apply changes
    active_point.x += position_active_point[0]
    active_point.y += position_active_point[1]
    active_point.bounce(WIDTH, HEIGHT)

    # Collision detection and resolution
    for other_point in points:
        if point != other_point and point.distance_to(other_point) < point.radius + other_point.radius:
            # Calculate angle of collision
            angle = math.atan2(other_point.y - point.y, other_point.x - point.x)

            # Push the points apart
            overlap = point.radius + other_point.radius - point.distance_to(other_point)
            point.x -= 0.1 * overlap * math.cos(angle)
            point.y -= 0.1 * overlap * math.sin(angle)
            other_point.x += 0.1 * overlap * math.cos(angle)
            other_point.y += 0.1 * overlap * math.sin(angle)

    # Teleporting walls
    for point in points:
        # After updating the position of the point, wrap it around to the other side of the screen if it goes off the edge
        point.x = point.x % WIDTH
        point.y = point.y % HEIGHT


        # Draw points
    for point in points:
        # If the point is either the active or target point, draw it with a green circle around it
        if point in [active_point, target_point]:
            point.draw(is_group=point in group_points, is_active_or_target=True)
        # If the point is a repelling point, draw it with a red circle around it
        elif point in repelling_points:
            point.draw(is_group=point in group_points, is_repelling=True)
        # Otherwise, just draw the point normally
        else:
            point.draw(is_group=point in group_points)

    # If lines are enabled
    if add_line:
        # Draw lines to represent repulsion
        for repelling_point in repelling_points:
            # Calculate the repulsion force between the active point and the current repelling point
            force, _ = repulsion_force(active_point, repelling_point)
            # Determine the width of the line based on the force
            line_width = max(1, min(int(force * 10), 5))       
            # Draw a red line from the active point to the repelling point
            pygame.draw.line(screen, (255, 0, 0), (active_point.x, active_point.y), (repelling_point.x, repelling_point.y), line_width)

    # If lines are enabled
    if add_line:
        # Draw a green line between target and active point
        # Calculate the repulsion force between the active point and the target point
        force, _ = attraction_force(active_point, target_point)
        # Determine the width of the line based on the force
        line_width2 = round(max(1, min(int(force * 10), 5)))
        # Draw a green line from the active point to the target point
        pygame.draw.line(screen, (0, 255, 0), (active_point.x, active_point.y), (target_point.x, target_point.y), line_width2)
 
        # Update the display
        pygame.display.flip()
    

    # Control the simulation speed
    pygame.time.Clock().tick(fps)

SystemExit: 